In [1]:
!pip install pandas

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)'))) - skipping


In [2]:
!pip install tqdm

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)'))) - skipping


In [3]:
!pip install matplotlib

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)'))) - skipping


In [4]:
import pandas as pd

In [5]:
pom_fn = "X:\\pomabuster\\pom\\pom2.jsonl"
arb_fn = "X:\\pomabuster\\arb\\arb2.jsonl"

pom_df = pd.read_json(pom_fn, lines=True)
arb_df = pd.read_json(arb_fn, lines=True)

In [6]:
pom_df.sample()

,block_number,transaction_hash,token_address,from_address,to_address,value,operator,recipient,is_trade
30183,13759385,0xc5c73aa14d84630308489dac661acb13b4041188acf1...,"[0xeb4c2781e4eba804ce9a9803c67d0893436bb27d, 0...","[0xd749bfe5de0326a0931e8b30bbb7e8ec91b47094, 0...","[0x220bda5c8994804ac96ebe4df184d25e5c2196d4, 0...","[397104056, 397104056, 397141302, 205623881371...",0xd749bfe5de0326a0931e8b30bbb7e8ec91b47094,0xd749bfe5de0326a0931e8b30bbb7e8ec91b47094,True


In [7]:
arb_df.sample()

,block_number,transaction_hash,token_address,from_address,to_address,value,operator,recipient,is_trade
150170,11952716,0x5d7729ac3661667fd33e04dcc62a6f677ed0266b23f1...,"[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","[0x0000000000007f150bd6f54c40a34d7c3d5e9f56, 0...","[0x349388dafb463ba2bc5cedc8ebc55c25d70f61bb, 0...","[950212720000000000, 979492626414188308673, 97...",0x0000000000007f150bd6f54c40a34d7c3d5e9f56,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,True


In [8]:
from dataclasses import dataclass

@dataclass
class Log:
    sender: str
    receiver: str
    amount: str
    asset: str
    idx: int
    
@dataclass
class Trade:
    operator: str
    recipient: str
    pool: str
    asset_in: str
    asset_out: str
    amount_in: str
    amount_out: str
    log_idx: list

In [9]:
pom_df.iloc[7097]

block_number                                                 16206489
transaction_hash    0x849776a908b33fd6fa74df9e3a51b89673373df18bae...
token_address       [0x8207c1ffc5b6804f6024322ccf34f29c3541ae26, 0...
from_address        [0x70bb8e6844dfb681810fd557dd741bcaf027bf94, 0...
to_address          [0xdcf0cb36e8e9d0607242e2e7babea5ba3395628f, 0...
value               [5626319686999554303963996, 260925895183678021...
operator                   0x70bb8e6844dfb681810fd557dd741bcaf027bf94
recipient                  0x3caca7b48d0573d793d3b0279b5f0029180e83b6
is_trade                                                         True
Name: 7097, dtype: object

In [10]:
def parse_record(record):
    assets = record.token_address
    senders = record.from_address
    receivers = record.to_address
    amounts = record.value
    logs = []
    for i in range(len(assets)):      
        log_dict = {
            "idx": i,
            "asset": assets[i],
            "sender": senders[i],
            "receiver": receivers[i],
            "amount": amounts[i],
        }
        logs.append(Log(**log_dict))
    return logs

In [11]:
pom_logs = parse_record(pom_df.iloc[0])
pom_assets = set([log.asset for log in pom_logs])
pom_block_number = pom_df.iloc[0]["block_number"]
pom_assets

{'0x03ab458634910aad20ef5f1c8ee96f1d6ac54919',
 '0x6b175474e89094c44da98b954eedeac495271d0f'}

In [12]:
arb = arb_df[(arb_df["block_number"] > pom_block_number) & (arb_df["block_number"] - pom_block_number <= 2)]
arb

,block_number,transaction_hash,token_address,from_address,to_address,value,operator,recipient,is_trade
488302,12465436,0x146d1444865b017ea9635ff1c0c7a7c15516fa409e30...,"[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","[0x0000000089341e263b85d84a0eea39f47c37a9d2, 0...","[0xc3d03e4f041fd4cd388c549ee2a29a9e5075882f, 0...","[93100000000000000000, 22188658395411476909038...",0x0000000089341e263b85d84a0eea39f47c37a9d2,0x0000000089341e263b85d84a0eea39f47c37a9d2,True


In [13]:
arb_logs = parse_record(arb.iloc[0])
arb_assets = [log.asset for log in arb_logs]
arb_assets

['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
 '0x6b175474e89094c44da98b954eedeac495271d0f',
 '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']

In [14]:
from tqdm import tqdm

link = []
for _, pom in tqdm(pom_df.iterrows()):
    pom_logs = parse_record(pom)
    pom_assets = set([log.asset for log in pom_logs])
    pom_block_number = pom["block_number"]
    arb = arb_df[(arb_df["block_number"] >= pom_block_number) & (arb_df["block_number"] - pom_block_number <= 2)]
    if not arb.empty:
        arb_logs = parse_record(arb.iloc[0])
        arb_assets = set([log.asset for log in arb_logs])
        if arb_assets.intersection(pom_assets):
            link.append((pom["transaction_hash"], arb.iloc[0]["transaction_hash"]))

58957it [02:20, 419.60it/s]


In [15]:
len(link)

16581

In [16]:
link_df = pd.DataFrame(link, columns=["pom_transaction_hash","arb_transaction_hash"])
link_df

,pom_transaction_hash,arb_transaction_hash
0,0x5dccf7377422eaa76613b770b088d7edf7c2bcfae00e...,0x146d1444865b017ea9635ff1c0c7a7c15516fa409e30...
1,0xca0c565f84ed6a3f984972aedaf74ebf98d173e950f2...,0x595c9b15fa0d2e20d6e9a2984db7689d18b40e316773...
2,0xaae9835fbdf73c594693d8c99ee6fd3a69de6c1a5244...,0x18945e46cd925cbe8b1c0f64ba852ffe06b7a9bbcbea...
3,0x99fa6f5af9c414d72554f2579f4e5239ede256c45a99...,0xfbe9dc34fd3d63a57b574c920fd223b71f2b06c3c9c1...
4,0xa22d3350fbde7901b3d355260ebb0a12b83ffb406bf6...,0x3645fdd74f85b482ee0e35cb677371edf8e4db3aa224...
...,...,...
16576,0x9d8a0bb73e81b747704d9fed5db85e58bd79e3b2ee1e...,0x726d90453615e5a0c8825d07870b06cbf99707bf05b7...
16577,0x754a9295ef700842916f8662c1cb7f4097404be61c6e...,0x6f2b13963fb9b295dd3069e4c5b4da612a0f5a85ed94...
16578,0xcf810cd9e2caf7e612f6fdea8333571c13a0d293d289...,0xe38f912c1333d5484f155766909223b18b0da10faf49...
16579,0x0468827952e016cf51e8a7ae047260e463e1acb80ff9...,0x0468827952e016cf51e8a7ae047260e463e1acb80ff9...


In [17]:
link_df[link_df["pom_transaction_hash"] == link_df["arb_transaction_hash"] ]

,pom_transaction_hash,arb_transaction_hash
5,0x0bb421acd8bf38f83121d50f0aaebc6ecca64cab1f71...,0x0bb421acd8bf38f83121d50f0aaebc6ecca64cab1f71...
7,0xbdefbcf724c2421cdb1e3f5cf9e05931d10db98bc41f...,0xbdefbcf724c2421cdb1e3f5cf9e05931d10db98bc41f...
9,0x4206204ab2197496ea8feb759def8fc311bf7361b4ca...,0x4206204ab2197496ea8feb759def8fc311bf7361b4ca...
11,0x8f977516ec35dd5333e4c42fd7ced553095424720265...,0x8f977516ec35dd5333e4c42fd7ced553095424720265...
12,0xa7889730f507afa57cb4911220dbdfa74d3b98dfd781...,0xa7889730f507afa57cb4911220dbdfa74d3b98dfd781...
...,...,...
16539,0x914000b6697018b8faa412f472330d03f824d230067f...,0x914000b6697018b8faa412f472330d03f824d230067f...
16546,0xd9c9990172eb10d466d5c21a3e5ec5983b7b60384cf2...,0xd9c9990172eb10d466d5c21a3e5ec5983b7b60384cf2...
16568,0x4cd635507dbe25eb6fad96e47f9cec3b87969be097ab...,0x4cd635507dbe25eb6fad96e47f9cec3b87969be097ab...
16574,0x67b85a0ff96f213f0fcfeb2a3f12bada530a9ff1d0e1...,0x67b85a0ff96f213f0fcfeb2a3f12bada530a9ff1d0e1...


In [18]:
pom_df.rename(columns={"transaction_hash": "pom_transaction_hash"}, inplace=True)
arb_df.rename(columns={"transaction_hash": "arb_transaction_hash"}, inplace=True)

In [19]:
result = pd.merge(pom_df, link_df, on="pom_transaction_hash")
result = pd.merge(result, arb_df, on="arb_transaction_hash")

In [20]:
result.sample()

,block_number_x,pom_transaction_hash,token_address_x,from_address_x,to_address_x,value_x,operator_x,recipient_x,is_trade_x,arb_transaction_hash,block_number_y,token_address_y,from_address_y,to_address_y,value_y,operator_y,recipient_y,is_trade_y
6695,11975148,0xa55cb57174d48e8dde5d9af1883f91246d7daeda3b43...,"[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","[0x2d3cd7b81c93f188f3cb8ad87c8acc73d6226e3a, 0...","[0x1c3132aeb09f5dd5e1d8283bd4f0363e7ea81c95, 0...","[200000000000000000000, 500000000000000000, 19...",0x2d3cd7b81c93f188f3cb8ad87c8acc73d6226e3a,0x0000000000000000000000000000000000000000,True,0x156f49eb8c3fe00580398042fc51dc9444b20dc6733a...,11975149,"[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","[0x0000000089341e263b85d84a0eea39f47c37a9d2, 0...","[0x7dd3f5705504002dc946aeafe6629b9481b72272, 0...","[1838670000000000000, 831664807871932454547, 1...",0x0000000089341e263b85d84a0eea39f47c37a9d2,0x0000000089341e263b85d84a0eea39f47c37a9d2,True


In [21]:
len(pd.merge(pom_df, link_df, on="pom_transaction_hash"))

16581

In [22]:
merged_pom_df = pd.merge(pom_df, link_df, on="pom_transaction_hash")

In [23]:
attacked_token = {}
for _, pom in tqdm(merged_pom_df.iterrows()):
    pom_logs = parse_record(pom)
    pom_assets = set([log.asset for log in pom_logs])
    for token in pom_assets:
        if token not in attacked_token:
            attacked_token[token] = 1
        else:
            attacked_token[token] += 1
attacked_token


16581it [00:01, 14848.97it/s]


{'0x6b175474e89094c44da98b954eedeac495271d0f': 3315,
 '0x03ab458634910aad20ef5f1c8ee96f1d6ac54919': 1334,
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': 6023,
 '0xdac17f958d2ee523a2206206994597c13d831ec7': 3350,
 '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2': 11997,
 '0x1f573d6fb3f13d689ff844b4ce37794d79a7ff1c': 866,
 '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599': 4115,
 '0xeb4c2781e4eba804ce9a9803c67d0893436bb27d': 3265,
 '0x57ab1ec28d129707052df4df418d58a2d46d5f51': 2452,
 '0x5e74c9036fb86bd7ecdcb084a0673efc32ea31cb': 568,
 '0x6f40d4a6237c257fff2db00fa0510deeecd303eb': 98,
 '0x96e61422b6a9ba0e068b6c5add4ffabc6a4aae27': 72,
 '0x44709a920fccf795fbc57baa433cc3dd53c44dbe': 3,
 '0x99d8a9c45b2eca8864373a26d1459e3dff1e17f3': 131,
 '0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b': 60,
 '0xe2f2a5c287993345a840db3b0845fbc70f5935a5': 275,
 '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984': 304,
 '0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5': 297,
 '0x6c8c6b02e7b2be14d4fa6022dfd6d75921d90e4e': 15,
 '0x0d877

In [24]:
sorted(attacked_token.items(), key=lambda x:x[1], reverse=True)[:5]

[('0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 11997),
 ('0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 6023),
 ('0x2260fac5e5542a773aa44fbcfedf7c193bc2c599', 4115),
 ('0xdac17f958d2ee523a2206206994597c13d831ec7', 3350),
 ('0x6b175474e89094c44da98b954eedeac495271d0f', 3315)]

In [25]:
len(sorted_attacked_token)

NameError: name 'sorted_attacked_token' is not defined

In [ ]:
import matplotlib.pyplot as plt
ax = pd.DataFrame.from_dict(sorted_attacked_token).plot.bar(rot=0, logy=True, legend=False, xlabel=None, xticks=[])
ax.set_xlabel('Assets')
ax.set_ylabel('Number of POMAs')
plt.tight_layout()
plt.savefig('../snp/asset_dist.pdf', format='pdf')

In [ ]:
sorted_attacked_token.plot.bar(rot=0)

In [ ]:
fn = "../tokens/scrape_erc20/erc20.jsonlines"
token_df = pd.read_json(fn, lines=True)
token_df.sample()

In [ ]:
import itertools

ZERO_ADDRESS = "0x0000000000000000000000000000000000000000"
ZERO_AMOUNT = "0"
def is_transfer_normal(log):
    return (log.sender != log.receiver 
        and log.sender != ZERO_ADDRESS
        and log.receiver != ZERO_ADDRESS
        and log.amount != ZERO_AMOUNT)

def is_transfer_minting(log):
    return (log.sender == ZERO_ADDRESS and log.amount != ZERO_AMOUNT)

def is_transfer_burning(log):
    return (log.receiver == ZERO_ADDRESS and log.amount != ZERO_AMOUNT)

def has_transfer_normal(logs):
    return [log for log in logs if is_transfer_normal(log)]

def has_transfer_minting(logs):
    return [log for log in logs if is_transfer_minting(log)]

def has_transfer_burning(logs):
    return [log for log in logs if is_transfer_burning(log)]

def has_liquidity_mining(logs):
    ts = has_transfer_normal(logs)
    tms = has_transfer_minting(logs)
    if not ts or not tms:
        return None
    for t, tm in pairwise(ts, tms):
        if t and tm and t.asset != tm.asset:
            trade_dict = {"operator": t.sender, 
                    "recipient": tm.receiver, 
                    "pool": t.receiver, 
                    "asset_in": t.asset,
                    "asset_out": tm.asset,
                    "amount_in": t.amount,
                    "amount_out": tm.amount,
                    "log_idx": [t.idx, tm.idx]}
            yield Trade(**trade_dict)
        return None

def has_liquidity_cancel(logs):
    ts = has_transfer_normal(logs)
    tbs = has_transfer_burning(logs)
    if not ts or not tbs:
        return None
    for t, tb in pairwise(ts, tbs):
        if t and tb and t.asset != tb.asset:
            trade_dict =  {"operator": tb.sender, 
                    "recipient": t.receiver, 
                    "pool": t.sender, 
                    "asset_in": tb.asset,
                    "asset_out": t.asset,
                    "amount_in": tb.amount,
                    "amount_out": t.amount,
                    "log_idx": [t.idx, tb.idx]}
            yield Trade(**trade_dict)
    return None

def pairwise(iterable_1, iterable_2=None):
    if iterable_2 is None:
        return itertools.product(iterable_1, iterable_1)
    return itertools.product(iterable_1, iterable_2)

def has_trade(logs):
    t = has_transfer_normal(logs)
    if not t:
        return None
    for t1, t2 in itertools.pairwise(t):
        if t1 and t2 and t1.asset != t2.asset and t1.receiver == t2.sender:
            trade_dict = {"operator": t1.sender, 
                    "recipient": t2.receiver, 
                    "pool": t1.receiver, 
                    "asset_in": t1.asset,
                    "asset_out": t2.asset,
                    "amount_in": t1.amount,
                    "amount_out": t2.amount,
                    "log_idx": [t1.idx, t2.idx]}
            yield Trade(**trade_dict)
    return None

def has_trade_alt(logs):
    lms = has_liquidity_mining(logs)
    lcs = has_liquidity_cancel(logs)
    if not lms or lcs:
        return None
    for lm, lc in itertools.pairwise(lms, lcs):
        if (lm and lc 
            and lm.recipient == lc.operator 
            and lm.asset_in != lc.asset_out 
            and lm.asset_out != lc.asset_in):
            trade_dict =  {"operator": lm.operator, 
                    "recipient": lc.recipient, 
                    "pool": lm.recipient, 
                    "asset_in": lm.asset_in,
                    "asset_out": lc.asset_out,
                    "amount_in": lm.amount_in,
                    "amount_out": lc.amount_out,
                    "log_idx": [lm.idx, lc.idx]}
            yield Trade(**trade_dict)
    return None

In [ ]:
def parse_record_y(record):
    assets = record.token_address_y
    senders = record.from_address_y
    receivers = record.to_address_y
    amounts = record.value_y
    logs = []
    for i in range(len(assets)):      
        log_dict = {
            "idx": i,
            "asset": assets[i],
            "sender": senders[i],
            "receiver": receivers[i],
            "amount": amounts[i],
        }
        logs.append(Log(**log_dict))
    return logs

In [ ]:
k = token_df.iloc[482]["address"]
token_df[token_df["address"]==k]["price_in_usd"].values[0]

In [ ]:
token_df[token_df["address"]==k]["decimals"].values[0]

In [ ]:
def calculate_profit(profit):
    total = 0
    for k, v in profit.items():
        value_in_usd = token_df[token_df["address"]==k]["price_in_usd"].values[0] if not token_df[token_df["address"]==k].empty else 0
        decimals = token_df[token_df["address"]==k]["decimals"].values[0]
        total += v * value_in_usd / 10**decimals
    return total

def profit_arbitrage(trades) -> bool:
    profit = {} # token => revenue
    for t in trades:
        if t.asset_in not in profit:
            profit[t.asset_in] = - int(t.amount_in)
        else:
            profit[t.asset_in] -= int(t.amount_in)
        if t.asset_out not in profit:
            profit[t.asset_out] = int(t.amount_out)
        else:
            profit[t.asset_out] += int(t.amount_out)
    return calculate_profit(profit)

In [ ]:
profit_by_address = []
for _, poma in tqdm(result.iterrows()):
    logs = parse_record_y(poma)
    trades = list(has_trade(logs))
    profit_by_address.append((poma["operator_x"], poma["operator_y"], profit_arbitrage(trades)))

In [ ]:
profit_df = pd.DataFrame([x[2] for x in profit_by_address], columns=["profit_in_usd"])

In [ ]:
result_profit_df = pd.concat([result, profit_df], axis=1)

In [ ]:
result_profit_df[result_profit_df["operator_x"] == result_profit_df["operator_y"]]

In [ ]:
max_poma = result_profit_df.sort_values(by="profit_in_usd").head()

In [ ]:
max_poma

In [ ]:
for _, poma in max_poma.iterrows():
    logs = parse_record_y(poma)
    trades = list(has_trade(logs))
    print(trades)

In [ ]:
token_df[token_df["address"] == "0xbbc2ae13b23d715c30720f079fcd9b4a74093505"]

In [ ]:
max_poma.values

In [ ]:
profit_by_address_df = pd.DataFrame(profit_by_address, columns=["address_x", "address_y", "profit_in_usd"])

In [ ]:
profit_by_address_df["profit_in_usd"].sum()

In [ ]:
sorted_profit_by_address_df = profit_by_address_df.groupby("address_x").sum().sort_values(by=['profit_in_usd'], ascending=False)

In [ ]:
ax = pd.DataFrame.from_dict(sorted_profit_by_address_df).plot.bar(rot=0, logy=True, legend=False, xlabel=None, xticks=[])
ax.set_xlabel('Operators')
ax.set_ylabel('USD')
plt.tight_layout()
plt.savefig('../snp/profit_dist.pdf', format='pdf')

In [ ]:
profit_by_address_df.groupby("address_y").sum().sort_values(by=['profit_in_usd'], ascending=False)

In [ ]:
profit_by_address_df.groupby("address_y").sum().sort_values(by=['profit_in_usd'], ascending=False).head(1)[]

In [ ]:
pomer_arbitrager_profit = profit_by_address_df.groupby(["address_x", "address_y"]).sum().sort_values(by=['profit_in_usd'], ascending=False)

In [ ]:
pomer_arbitrager_profit[pomer_arbitrager_profit["address_x"] != pomer_arbitrager_profit["address_y"]]